# DuckDB로 하는 암호화폐 데이터 분석

이 노트북은 DuckDB를 사용해 parquet 파일에 저장된 OHLCV 데이터를 SQL로 분석합니다.

DuckDB는 파일 기반 분석용 인메모리 SQL 엔진으로,  
parquet 파일을 별도 import 없이 직접 쿼리할 수 있습니다.

## 다루는 SQL 개념

| 섹션 | SQL 개념 |
|------|----------|
| 1. 기본 조회 | `SELECT`, `WHERE`, `ORDER BY`, `LIMIT` |
| 2. 집계 분석 | `GROUP BY`, `AVG`, `MAX`, `MIN`, `SUM` |
| 3. 윈도우 함수 | `LAG()`, `AVG() OVER`, `ROWS BETWEEN` |
| 4. CTE | `WITH` 절로 복잡한 쿼리 단계 분리 |
| 5. 멀티 티커 | `JOIN`, `CORR`, `STDDEV` |

In [ ]:
import os
from pathlib import Path

import duckdb
import matplotlib.pyplot as plt
import pandas as pd

# DuckDB 인메모리 연결 — 파일 없이 바로 parquet을 쿼리
con = duckdb.connect()

# 데이터 경로
DATA_DIR = Path("../data/upbit")

# 사용 가능한 parquet 파일 목록
files = sorted(DATA_DIR.glob("*.parquet"))
for f in files:
    print(f.name)

## 1. 기본 조회

```sql
SELECT 컬럼  FROM 파일경로  WHERE 조건  ORDER BY 정렬  LIMIT 행수
```

DuckDB는 `FROM 'path/to/file.parquet'` 처럼 파일 경로를 직접 쓸 수 있습니다.

In [ ]:
# 스키마 확인 — 어떤 컬럼이 있는지 먼저 파악
con.execute("DESCRIBE SELECT * FROM '../data/upbit/KRW-BTC_day.parquet'").df()

In [ ]:
# 최근 10일 BTC 일봉 데이터
con.execute("""
    SELECT
        date,
        open,
        high,
        low,
        close,
        ROUND(volume, 2) AS volume
    FROM '../data/upbit/KRW-BTC_day.parquet'
    WHERE date >= '2024-01-01'
    ORDER BY date DESC
    LIMIT 10
""").df()

## 2. 집계 함수

`GROUP BY`로 기간별 요약 통계를 구합니다.  
`strftime(date, '%Y-%m')`은 날짜를 `'2024-03'` 형식의 월 문자열로 변환합니다.

In [ ]:
# 월별 BTC 요약 통계
monthly = con.execute("""
    SELECT
        strftime(date, '%Y-%m')  AS month,
        ROUND(AVG(close), 0)     AS avg_close,    -- 월평균 종가
        ROUND(MAX(high), 0)      AS max_high,     -- 월중 최고가
        ROUND(MIN(low), 0)       AS min_low,      -- 월중 최저가
        ROUND(
            (MAX(high) - MIN(low)) / MIN(low) * 100, 2
        )                        AS range_pct,    -- 월간 고저 변동폭 %
        ROUND(SUM(volume), 0)    AS total_volume  -- 월간 거래량 합계
    FROM '../data/upbit/KRW-BTC_day.parquet'
    WHERE date >= '2023-01-01'
    GROUP BY month
    ORDER BY month DESC
    LIMIT 24
""").df()

monthly.head(12)

In [ ]:
# 월별 평균 종가 차트
plot_df = monthly.sort_values("month")
fig, ax = plt.subplots(figsize=(12, 4))
ax.bar(plot_df["month"], plot_df["avg_close"] / 1e6, color="steelblue", alpha=0.8)
ax.set_xlabel("월")
ax.set_ylabel("평균 종가 (백만 원)")
ax.set_title("BTC 월별 평균 종가")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. 윈도우 함수

윈도우 함수는 행을 그룹으로 집계하지 않고,  
각 행에 대해 **주변 행의 값을 참조**할 수 있습니다.

```sql
-- LAG(컬럼, n): n행 이전 값 참조 (전일 종가 등)
LAG(close, 1) OVER (ORDER BY date)

-- AVG() OVER: 이동평균
AVG(close) OVER (ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW)
--                                           ↑ 현재 행 기준 30행(30일) 슬라이딩 윈도우
```

In [ ]:
# 일간 수익률 + 이동평균 (MA7, MA30)
df_ma = con.execute("""
    SELECT
        date,
        close,

        -- LAG(): 전일 종가를 현재 행에서 참조
        LAG(close, 1) OVER (ORDER BY date)  AS prev_close,

        -- 일간 수익률 = (당일 종가 - 전일 종가) / 전일 종가 * 100
        ROUND(
            (close - LAG(close, 1) OVER (ORDER BY date))
            / LAG(close, 1) OVER (ORDER BY date) * 100,
        2)                                   AS daily_ret_pct,

        -- 7일 이동평균
        ROUND(AVG(close) OVER (
            ORDER BY date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ), 0)                                AS ma_7,

        -- 30일 이동평균
        ROUND(AVG(close) OVER (
            ORDER BY date
            ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
        ), 0)                                AS ma_30

    FROM '../data/upbit/KRW-BTC_day.parquet'
    WHERE date >= '2023-01-01'
    ORDER BY date
""").df()

df_ma.tail(10)

In [ ]:
# 종가 + 이동평균 차트
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(13, 7), sharex=True)

ax1.plot(df_ma["date"], df_ma["close"] / 1e6, lw=1, label="종가", color="gray", alpha=0.6)
ax1.plot(df_ma["date"], df_ma["ma_7"] / 1e6, lw=1.5, label="MA7", color="steelblue")
ax1.plot(df_ma["date"], df_ma["ma_30"] / 1e6, lw=2, label="MA30", color="tomato")
ax1.set_ylabel("종가 (백만 원)")
ax1.set_title("BTC 종가 및 이동평균")
ax1.legend()

colors = df_ma["daily_ret_pct"].apply(lambda x: "tomato" if x < 0 else "steelblue")
ax2.bar(df_ma["date"], df_ma["daily_ret_pct"], color=colors, alpha=0.7, width=0.8)
ax2.axhline(0, color="black", lw=0.8)
ax2.set_ylabel("일간 수익률 (%)")
ax2.set_xlabel("날짜")

plt.tight_layout()
plt.show()

## 4. CTE — 변동성 돌파 시그널을 SQL로 구현

CTE(`WITH` 절)는 복잡한 쿼리를 **이름 붙인 중간 단계**로 나눠  
가독성과 재사용성을 높입니다.

```sql
WITH 단계1 AS (
    SELECT ...
),
단계2 AS (
    SELECT ... FROM 단계1   -- 앞 단계를 테이블처럼 참조
)
SELECT * FROM 단계2
```

VBO 진입 조건: `당일 고가 ≥ 시가 + 전일 레인지 × noise_ratio`

In [ ]:
NOISE_RATIO = 0.6

df_vbo = con.execute(f"""
    -- 1단계: 전일 레인지 계산
    WITH base AS (
        SELECT
            date,
            open,
            high,
            low,
            close,
            LAG(high - low, 1) OVER (ORDER BY date) AS prev_range
        FROM '../data/upbit/KRW-BTC_day.parquet'
    ),

    -- 2단계: 목표가 및 진입 시그널
    signals AS (
        SELECT
            date,
            open,
            high,
            close,
            prev_range,
            ROUND(open + prev_range * {NOISE_RATIO}, 0)  AS target_price,
            high >= open + prev_range * {NOISE_RATIO}     AS entry_signal
        FROM base
        WHERE prev_range IS NOT NULL
    ),

    -- 3단계: 시그널 발생일에 예상 수익률 계산
    -- 진입가 = 목표가, 청산가 = 다음날 시가 (VBODayExit 방식)
    trades AS (
        SELECT
            date,
            target_price                         AS entry_price,
            LEAD(open, 1) OVER (ORDER BY date)   AS exit_price,  -- LEAD(): 다음 행 참조
            close,
            entry_signal
        FROM signals
        WHERE entry_signal = TRUE
    )

    SELECT
        date,
        entry_price,
        exit_price,
        ROUND((exit_price - entry_price) / entry_price * 100, 2) AS ret_pct
    FROM trades
    WHERE exit_price IS NOT NULL
    ORDER BY date DESC
    LIMIT 20
""").df()

df_vbo.head(10)

In [ ]:
# 시그널 통계 요약
con.execute(f"""
    WITH base AS (
        SELECT
            date,
            open,
            high,
            low,
            close,
            LAG(high - low, 1) OVER (ORDER BY date) AS prev_range
        FROM '../data/upbit/KRW-BTC_day.parquet'
    ),
    signals AS (
        SELECT
            date,
            open + prev_range * {NOISE_RATIO}         AS target_price,
            high >= open + prev_range * {NOISE_RATIO}  AS entry_signal
        FROM base
        WHERE prev_range IS NOT NULL
    ),
    trades AS (
        SELECT
            date,
            target_price,
            LEAD(open, 1) OVER (ORDER BY date) AS exit_price,
            entry_signal
        FROM signals
        WHERE entry_signal = TRUE
    )
    SELECT
        COUNT(*)                                           AS total_trades,
        SUM(CASE WHEN exit_price > target_price THEN 1 ELSE 0 END) AS wins,
        ROUND(
            SUM(CASE WHEN exit_price > target_price THEN 1 ELSE 0 END)
            * 100.0 / COUNT(*), 1
        )                                                  AS win_rate_pct,
        ROUND(AVG((exit_price - target_price) / target_price * 100), 3) AS avg_ret_pct,
        ROUND(MAX((exit_price - target_price) / target_price * 100), 2) AS best_trade_pct,
        ROUND(MIN((exit_price - target_price) / target_price * 100), 2) AS worst_trade_pct
    FROM trades
    WHERE exit_price IS NOT NULL
""").df()

## 5. 멀티 티커 분석 — BTC / ETH 상관관계

`JOIN`으로 두 티커 데이터를 날짜 기준으로 합쳐  
상관관계와 연간 변동성을 계산합니다.

```sql
-- USING (date): 두 테이블에 같은 이름의 컬럼(date)으로 JOIN
FROM btc b JOIN eth e USING (date)

-- CORR(): 피어슨 상관계수 (−1 ~ 1)
-- STDDEV() * SQRT(365): 일간 표준편차 → 연간 변동성으로 환산
```

In [ ]:
# BTC / ETH 상관관계 및 연간 변동성
con.execute("""
    WITH btc AS (
        SELECT
            date,
            close / LAG(close, 1) OVER (ORDER BY date) - 1 AS btc_ret
        FROM '../data/upbit/KRW-BTC_day.parquet'
    ),
    eth AS (
        SELECT
            date,
            close / LAG(close, 1) OVER (ORDER BY date) - 1 AS eth_ret
        FROM '../data/upbit/KRW-ETH_day.parquet'
    ),
    joined AS (
        SELECT b.date, b.btc_ret, e.eth_ret
        FROM btc b
        JOIN eth e USING (date)
        WHERE b.btc_ret IS NOT NULL AND e.eth_ret IS NOT NULL
    )
    SELECT
        COUNT(*)                                       AS days,
        ROUND(CORR(btc_ret, eth_ret), 4)               AS btc_eth_corr,
        ROUND(STDDEV(btc_ret) * SQRT(365) * 100, 2)   AS btc_ann_vol_pct,
        ROUND(STDDEV(eth_ret) * SQRT(365) * 100, 2)   AS eth_ann_vol_pct,
        ROUND(AVG(btc_ret) * 365 * 100, 2)            AS btc_ann_ret_pct,
        ROUND(AVG(eth_ret) * 365 * 100, 2)            AS eth_ann_ret_pct
    FROM joined
""").df()

In [ ]:
# 연도별 상관관계 추이 — 시장 국면에 따라 상관관계가 어떻게 변하는지 확인
con.execute("""
    WITH btc AS (
        SELECT
            date,
            close / LAG(close, 1) OVER (ORDER BY date) - 1 AS btc_ret
        FROM '../data/upbit/KRW-BTC_day.parquet'
    ),
    eth AS (
        SELECT
            date,
            close / LAG(close, 1) OVER (ORDER BY date) - 1 AS eth_ret
        FROM '../data/upbit/KRW-ETH_day.parquet'
    ),
    joined AS (
        SELECT b.date, b.btc_ret, e.eth_ret
        FROM btc b
        JOIN eth e USING (date)
        WHERE b.btc_ret IS NOT NULL AND e.eth_ret IS NOT NULL
    )
    SELECT
        strftime(date, '%Y')                           AS year,
        COUNT(*)                                       AS days,
        ROUND(CORR(btc_ret, eth_ret), 4)               AS btc_eth_corr,
        ROUND(STDDEV(btc_ret) * SQRT(365) * 100, 1)   AS btc_vol_pct,
        ROUND(STDDEV(eth_ret) * SQRT(365) * 100, 1)   AS eth_vol_pct
    FROM joined
    GROUP BY year
    ORDER BY year
""").df()

## 6. 노이즈 비율별 전략 성과 비교

VBO의 핵심 파라미터인 `noise_ratio`를 0.3 ~ 0.8 범위에서 SQL로 비교합니다.  
Python 루프와 DuckDB 쿼리를 조합해 파라미터 스윕을 수행합니다.

In [ ]:
def vbo_stats_sql(noise_ratio: float) -> dict:
    """noise_ratio별 VBO 전략 통계를 SQL로 계산."""
    row = con.execute(f"""
        WITH base AS (
            SELECT
                date,
                open,
                high,
                LAG(high - low, 1) OVER (ORDER BY date) AS prev_range
            FROM '../data/upbit/KRW-BTC_day.parquet'
        ),
        trades AS (
            SELECT
                open + prev_range * {noise_ratio}                        AS entry,
                LEAD(open, 1) OVER (ORDER BY date)                       AS exit_p
            FROM base
            WHERE prev_range IS NOT NULL
              AND high >= open + prev_range * {noise_ratio}
        )
        SELECT
            {noise_ratio}                                                   AS noise_ratio,
            COUNT(*)                                                        AS trades,
            ROUND(AVG((exit_p - entry) / entry * 100), 3)                  AS avg_ret_pct,
            ROUND(
                SUM(CASE WHEN exit_p > entry THEN 1 ELSE 0 END)
                * 100.0 / COUNT(*), 1
            )                                                               AS win_rate_pct
        FROM trades
        WHERE exit_p IS NOT NULL
    """).fetchone()
    return {"noise_ratio": row[0], "trades": row[1], "avg_ret_pct": row[2], "win_rate_pct": row[3]}


import numpy as np

results = [vbo_stats_sql(r) for r in np.arange(0.3, 0.9, 0.1).round(1)]
df_sweep = pd.DataFrame(results)
df_sweep

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(13, 4))

ax1.bar(df_sweep["noise_ratio"].astype(str), df_sweep["trades"], color="steelblue", alpha=0.8)
ax1.set_title("noise_ratio별 거래 횟수")
ax1.set_xlabel("noise_ratio")
ax1.set_ylabel("횟수")

ax2.bar(df_sweep["noise_ratio"].astype(str), df_sweep["avg_ret_pct"], color="tomato", alpha=0.8)
ax2.set_title("noise_ratio별 평균 수익률")
ax2.set_xlabel("noise_ratio")
ax2.set_ylabel("평균 수익률 (%)")
ax2.axhline(0, color="black", lw=0.8)

ax3.bar(df_sweep["noise_ratio"].astype(str), df_sweep["win_rate_pct"], color="seagreen", alpha=0.8)
ax3.set_title("noise_ratio별 승률")
ax3.set_xlabel("noise_ratio")
ax3.set_ylabel("승률 (%)")
ax3.axhline(50, color="gray", lw=0.8, linestyle="--")

plt.suptitle("VBO 파라미터 스윕 — BTC (전체 기간)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()